구조적 API는 **데이터 흐름을 정의하는 기본 추상화 개념**이다.

구조적 API는 다음의 다양한 데이터를 처리할 수 있다.

* 비정형 로그파일
* 반정형 csv파일
* 정형적인 파케이(parquet) 파일

※ 여기서 정형/반정형/비정형 데이터의 정의를 한번 짚고 넘어가자. [출처](https://computer-science-student.tistory.com/471)

* **정형 데이터(Structured Data)**

    정형 데이터는 **구조화된 데이터 혹은 미리 정해진 구조에 따라 저장된 데이터**이다. 표 안에서 행과 열에 의해 지정된 각 칸에 데이터를 저장하는 csv파일등이 대표적인 예다. <br/>
    
    
* **반정형 데이터(Semi-Structured Data)**

    반정형 데이터는 구조에 따라 저장된 데이터지만 **정형 데이터와 달리 데이터 내용 안에 구조에 대한 설명이 함께 존재**한다. 그렇기 때문에 데이터 내용에 대한 설명인 구조를 파악하는 파싱 과정이 필요하고, 보통 파일 형태로 저장된다. 반정형 데이터의 예시로 웹에서 데이터를 교환하기 위해 작성하는 HTML, XML, JSON 문서나 웹 로그, 센서 데이터 등이 있다.  <br/>
    
    
* **비정형 데이터(Unstructured Data)**

    비정형 데이터는 **정해진 구조가 없이 저장된 데이터**다. 소셜 데이터의 텍스트, 이미지, 영상, 워드나 PDF 문서와 같은 멀티미디어 데이터가 대표적인 예다. <br/>
    
구조적 API는 다음의 3가지 분산 컬렉션 API가 있다.

* Dataset
* DataFrame
* SQL 테이블과 뷰

배치와 스트리밍 처리에서 구조적 API를 사용할 수 있다. 


# 1. DataFrame과 Dataset
---

DataFrame과 Dataset은 잘 정의된 로우와 컬럼을 갖는 분산 테이블 형태의 컬렉션이다. DataFrame과 Dataset은 **결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야 하는지 정의하는 지연 연산의 실행 계획이고 불변성을 갖는다.**

<BR/><BR/>

# 2. 스키마
---

스키마는 **DataFrame의 컬럼명과 데이터 타입을 정의**한다. 스키마는 데이터소스에서 얻거나(스키마 온 리드) 직접 정의할 수 있다. 

<br/><br/>
# 3. 스파크의 구조적 데이터 타입 개요
---

**스파크는 일종의 프로그래밍 언어라고 생각하면 좋다.** 스파크는 실행 계획 수립과 처리에 사용하는 자체 데이터 타입 정보를 갖는 **카탈리스트 엔진**을 사용한다. 이것은 다양한 실행 최적화 기능을 제공한다. 파이썬에서 스파크의 구조적 API를 사용하면 스파크의 데이터 타입을 사용한다. 예시를 보자. 이 예시는 스파크의 덧셈 연산을 수행한다.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("Section3").\
config("spark.some.config.option","some-value")\
.getOrCreate()

df = spark.range(500).toDF("number")
df.select(df["number"]+10)

DataFrame[(number + 10): bigint]

스파크에서 덧셈 연산이 수행되는 이유는 **스파크가 지원하는 언어(파이썬)를 이용해 작성된 표현식을 카탈리스크 엔진에서 스파크의 데이터 타입으로 변환해 명령을 처리**하기 때문이다. 

## 1) DataFrame과 Dataset 비교

구조적 API에는 '비타입형'인 DataFrame과 '타입형'인 Dataset이 있다. 엄연히 따지자면 DataFrame도 데이터 타입이 있지만, 스키마에 명시된 데이터 타입의 일치 여부를 런타임이 되어서야 확인한다. DataFrame를 파이썬에서 지원하기 때문에, 이것을 위주로 보자.

스파크의 DataFrame은 Row 타입으로 구성된 Dataset이다. Row 타입은  스파크가 사용하는 '연산에 최적화된 인메모리 포맷'의 내부적인 표현 방식이다.  자체 데이터 포맷을 사용하기 때문에 효율적인 연산이 가능하다.

**당장 기억할 것은 DataFrame을 사용하면 스파크의 최적화된 내부 포맷을 사용할 수 있다는 것이다.**

## 2) 컬럼

컬럼은 다음의 타입들을 표현한다.
* 단순 데이터 타입 : 정수형,문자열..
* 복합 데이터 타입 : 배열,맵..
* null 값

## 3) 로우

로우는 데이터 레코드이다. 이후로 대부분의 영어들은 다 소문자화해서 쓰겠다.. dataframe의 레코드는 row 타입으로 구성된다. 

In [6]:
spark.range(2).collect() # collect()는 모든 레코드를 row 리스트로 반환하는 액션 메소드이다.

[Row(id=0), Row(id=1)]

## 4) 스파크 데이터 타입

스파크는 여러가지 내부 데이터 타입을 갖고 있다. 책 115 ~ 118 page를 한번 봐보자.

<br/><br/>

# 4. 구조적 API의 실행 과정

이 절은 스파크 코드가 클러스터에 가서 처리되는 과정을 설명한다. 꼭 잘 이해해야한다.

그림과 같이 봐야해서, 이 부분도 책을 보는 것이 좋을 것 같다. 책 119~121 page를 한번 봐보자.
